In [100]:
import pandas as pd
import os
#https://github.com/sean5446/icd9scraper

In [138]:
# For file in os.listdir('data/raw/'):
MAX_TABS = 3

for file in os.listdir("../data/raw/icd9"):
    if file.startswith(("cleaned", "dataframe")):
        continue
    
    with open(f"../data/raw/icd9/{file}", "r") as f:
        lines = f.readlines()
    
    with open(f"../data/raw/icd9/cleaned_{file}", "w") as f:
        for line in lines:
            tab_diff = MAX_TABS - line.count("\t")
            if tab_diff > 0:
                line = line.strip("\n") + "\t" * tab_diff + "\n"
            f.write(line)
            
    df = pd.read_csv(f"../data/raw/icd9/cleaned_{file}", sep="\t", header=None, usecols= [0,1,2], names=["chapter_codes", "block_codes", "category_codes"])
    if df.chapter_codes.nunique() == 1:
        if df.category_codes.isnull().all():
            df.category_codes = df.block_codes
        df[["chapter_codes", "block_codes"]] = df[["chapter_codes", "block_codes"]].ffill()
        df = df.dropna().reset_index(drop=True)
        df[["chapter_codes", "chapter_des"]] = df["chapter_codes"].str.split(" ", n=1, expand=True)
        df[["block_codes", "block_des"]] = df["block_codes"].str.split(" ", n=1, expand=True)
        df[["category_codes", "category_des"]] = df["category_codes"].str.split(" ", n=1, expand=True)
        df.to_csv(f"../data/raw/icd9/dataframe_{file.strip('.txt')}.csv", index=False)
    else:
        print("Problem", file)

In [149]:
df = pd.DataFrame()
for file in os.listdir("../data/raw/icd9"):
    if file.startswith("dataframe"):
        df_part = pd.read_csv(f"../data/raw/icd9/{file}")
        df = pd.concat([df, df_part], ignore_index=True)
df.to_csv("../data/interim/icd9_codes_and_des.csv", index=False)

In [145]:
df.shape

(123, 6)

In [94]:
df.chapter_codes.unique()

array(['001-139 Infectious And Parasitic Diseases'], dtype=object)

In [53]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re
import unicodedata

In [69]:
NHS_BASE = "https://www.nhs.uk/conditions/"

def get_symptoms_from_nhs(url: str) -> str | None:
    """
    Load symptoms from NHS

    Parameters
    ----------
    url : str
        URL to NHS page

    Returns
    -------
    str | None
        Either a string with symptoms, Message with error or None
    """
    res = requests.get(url)
    if res.status_code in range(200, 300):
        soup = BeautifulSoup(
            res.content,
            "lxml",
            parse_only=SoupStrainer("section"),
        )
        symptoms_list_raw = soup.findAll("h2")
        for item in soup.find_all("h2"):
            if "Symptoms" not in item.text:
                continue
            else:
                symptoms_list = item.find_next_siblings("ul")[0]
                text = ", ".join([li.text for li in symptoms_list.find_all("li")])
                text_cleaned = unicodedata.normalize("NFKD", text)
                return text_cleaned
        else:
            return f"No Value"
    return None

In [72]:
get_symptoms_from_nhs("https://www.nhs.uk/conditions/Histoplasmosis/")